In [ ]:
import pandas as pd
import numpy as np
import folium
import json
import matplotlib.pyplot as pp
%matplotlib inline

from IPython.display import display, HTML

We use pre-cleaned dataset, i.e. we already got rid of useless rows and columns but did not perform any other processing.

In [ ]:
-

In [ ]:
cantons_b = json.load(open("topojson/ch-cantons.topojson.json"))

In [ ]:
m = folium.Map([46.90, 8.50], zoom_start=8)

m.choropleth(geo_data=cantons_b,
             threshold_scale=[0., 1.5, 3.8, 4.5, 5],                   # Compute automaticall ? see next cell
             topojson="objects.cantons", 
             data=ur["rate"].reset_index(level=[0,1,2], drop=True), 
             fill_color='OrRd', 
             key_on="id",
             legend_name="Unemployment Rate")
m

In [ ]:
pp.plot(ur["rate"], np.zeros_like(ur["rate"]), 'x')
pp.show()                                           # Shall we automate this or is it good enough ?

In [ ]:
# Second map of pt. 2 : the rate is already computed this way... what shall we do ?

In [ ]:
m = folium.Map([46.90, 8.50], zoom_start=8)
pp.plot(ur["rate_nat_diff"], np.zeros_like(ur["rate_nat_diff"]), 'x')
pp.show()        

m.choropleth(geo_data=cantons_b,
             threshold_scale=[0., 1.1, 2.2, 3.2, 4,5.5],                   # Compute automaticall ? see next cell
             topojson="objects.cantons", 
             data=ur["rate_nat_diff"].reset_index(level=[0,1,2], drop=True), 
             fill_color='OrRd', 
             key_on="id",
             legend_name="Unemployment Rates difference")
m